In [ ]:
import pandas as pd
import os
import glob
import numpy as np
import seaborn as sns
import matplotlib.pylab as plt 
import numpy as np
import math



In [ ]:
csv_files = glob.glob(os.path.join(os.getcwd() + r"\data\\fixed", "*csv"))
print(f"csv files: {csv_files}")

In [ ]:
def merge_events(csv_files, index1, index2):
    # merge csv files
    df1 = pd.read_csv(csv_files[index1], index_col=0)
    df2 = pd.read_csv(csv_files[index2], index_col=0)
    
    df = pd.merge(df1, df2, on='user_id', how='left')
    return df
    

In [ ]:
df_event_a = merge_events(csv_files, 0, 3)
df_event_b = merge_events(csv_files, 1, 4)
df_event_before = merge_events(csv_files, 2, 5)

In [ ]:
# Order_value, which is our only numeric data.
# Clearly we can see that there is outliers, Min value in both Event B and Event Before is negative.
print(f"Event a {df_event_a.describe()}")
print("")
print(f"Event b {df_event_b.describe()}")
print("")
print(f"Event before {df_event_before.describe()}")

### Counting values on each column to see where we have null-values

In [ ]:
df_event_a.info()

In [ ]:
df_event_b.info()

In [ ]:
df_event_before.info()

In all three cases we see around 20 % missing values in country and gender.

# EDA

Time to see where we have outliers

In [ ]:
sns.boxplot(df_event_a)
plt.show()
sns.boxplot(df_event_b)
plt.show()
sns.boxplot(df_event_before)

We can clearly see outliers both high and low values. But values below zero must be an error so only these i will remove.

In [ ]:
def remove_below_zero_outliers(df, df_name):
    # function to remove values below or equal to zero
    print(f"Outliers removed from {df_name}: {df.order_value[df.order_value <= 0].count()}")
    df_filtered = df.loc[(df['event_type'] != 'purchased_shopping_cart') | (df['order_value'] > 0)]

    df_filtered = df_filtered.drop(df_filtered.loc[(df_filtered['event_type'] == 'purchased_shopping_cart') & (df_filtered['order_value'] < 0)].index)
    
    return df_filtered

In [ ]:
df_event_a = remove_below_zero_outliers(df_event_a, "df_event_a")
df_event_b = remove_below_zero_outliers(df_event_b, "df_event_b")
df_event_before = remove_below_zero_outliers(df_event_before, "df_event_before")

## Time frame

In [ ]:
# Change type of data from object to datetime
df_event_a.date = pd.to_datetime(df_event_a.date)
df_event_b.date = pd.to_datetime(df_event_b.date)
df_event_before.date = pd.to_datetime(df_event_before.date)

In [ ]:
df_event_a['weekday'] = df_event_a.date.dt.day_name()
df_event_b['weekday'] = df_event_b.date.dt.day_name()
df_event_before['weekday'] = df_event_before.date.dt.day_name()

In [ ]:
print(f"Event before: {df_event_before.date.min()} - {df_event_before.date.max()}")
print(f"Event A before: {df_event_a.date.min()} - {df_event_a.date.max()}")
print(f"Event B before: {df_event_b.date.min()} - {df_event_b.date.max()}")

Event A and Event B rund som 1st of February to 7th of February. Event Before is the whole of January

# Conversion

In [ ]:
conv_event_a = [df_event_a.event_type[df_event_a.event_type == "opened_shopping_cart"].count(),
    df_event_a.event_type[df_event_a.event_type == "purchased_shopping_cart"].count()]
conv_event_b = [df_event_b.event_type[df_event_b.event_type == "opened_shopping_cart"].count(),
    df_event_b.event_type[df_event_b.event_type == "purchased_shopping_cart"].count()]

conv_event_before = [df_event_before.event_type[df_event_before.event_type == "opened_shopping_cart"].count(),
    df_event_before.event_type[df_event_before.event_type == "purchased_shopping_cart"].count()]

print(f"Event A: {conv_event_a} {(conv_event_a[1]/conv_event_a[0])*100} %")
print(f"Event B: {conv_event_b} {(conv_event_b[1]/conv_event_b[0])*100} %")
print(f"Event Before: {conv_event_before} {round((conv_event_before[1]/conv_event_before[0])*100,1)} %")

In [ ]:
print(f"Difference betweem Event B and Event A: {round((conv_event_b[1]-conv_event_a[1]) / conv_event_a[1],2)*100} %")
print(f"Difference betweem Event B and Event Before: {round((((conv_event_b[1]/conv_event_b[0]))-((conv_event_before[1]/conv_event_before[0])))/((conv_event_before[1]/conv_event_before[0]))*100,1)} %")

## Sample size

In [ ]:
# N = population size - number of conversion in Event Before
N = conv_event_before[0]
# z = z-score
z = 1.96
# e = margin of error
e = 0.05
# p = standard deviation
p = 0.5

# numerator or Unlimited population
numerator = ((z**2) * p * (1 - p)) / (e**2)

denominator = 1 + (((z**2) * p * (1 - p)) / (e**2 * N))

sample_size = math.ceil(numerator / denominator)

print(f"Sample size with unlimited population {round(numerator)}")
print(f"Sample size with finite population: {sample_size}")

In [ ]:
https://www.calculator.net/sample-size-calculator.html

In [ ]:
print(f"Number of conversions for Event a: {conv_event_a[1]}")
print(f"Number of conversions for Event b: {conv_event_b[1]}")


The number of samples are to low to make a statistically correct judgment on the data.

# Gender split

In [ ]:
#Event A:
event_a_gender_opened = df_event_a.gender[df_event_a.event_type=="opened_shopping_cart"].value_counts(normalize=True)*100
event_a_gender_purch = df_event_a.gender[df_event_a.event_type=="purchased_shopping_cart"].value_counts(normalize=True)*100
#Event B:
event_b_gender_opened = df_event_b.gender[df_event_b.event_type=="opened_shopping_cart"].value_counts(normalize=True)*100
event_b_gender_purch = df_event_b.gender[df_event_b.event_type=="purchased_shopping_cart"].value_counts(normalize=True)*100
#Event Before:
event_be_gender_opened = df_event_before.gender[df_event_before.event_type=="opened_shopping_cart"].value_counts(normalize=True)*100
event_be_gender_purch = df_event_before.gender[df_event_before.event_type=="purchased_shopping_cart"].value_counts(normalize=True)*100


In [ ]:
gender_data = pd.DataFrame({
    'Event_A_Opened': [round(event_a_gender_opened[0],1), round(event_a_gender_opened[1],1), round(event_a_gender_opened[2],1)],
    'Event_B_Opened': [round(event_b_gender_opened[0],1), round(event_b_gender_opened[1],1), round(event_b_gender_opened[2],1)],
    'Event_Before_Opened' : [round(event_be_gender_opened[0],1), round(event_be_gender_opened[1],1), round(event_be_gender_opened[2],1)], 
    'Event_A_Purchase': [round(event_a_gender_purch[0],1), round(event_a_gender_purch[1],1), round(event_a_gender_purch[2],1)],
    'Event_B_Purchase': [round(event_b_gender_purch[0],1), round(event_b_gender_purch[1],1), round(event_b_gender_purch[2],1)],
    'Event_Before_Purchase' : [round(event_be_gender_purch[0],1), round(event_be_gender_purch[1],1), round(event_be_gender_purch[2],1)] 
}, index=['man', 'woman', 'non-binary'])
gender_data

## Country split

In [ ]:
#Event A:
event_a_country_opened = df_event_a.country[df_event_a.event_type=="opened_shopping_cart"].value_counts(normalize=True)*100
event_a_country_purch = df_event_a.country[df_event_a.event_type=="purchased_shopping_cart"].value_counts(normalize=True)*100
#Event B:
event_b_country_opened = df_event_b.country[df_event_b.event_type=="opened_shopping_cart"].value_counts(normalize=True)*100
event_b_country_purch = df_event_b.country[df_event_b.event_type=="purchased_shopping_cart"].value_counts(normalize=True)*100
#Event Before:
event_be_country_opened = df_event_before.country[df_event_before.event_type=="opened_shopping_cart"].value_counts(normalize=True)*100
event_be_country_purch = df_event_before.country[df_event_before.event_type=="purchased_shopping_cart"].value_counts(normalize=True)*100


In [ ]:

df_country = pd.concat([round(event_a_country_purch,1), round(event_b_country_purch,1), round(event_be_country_purch,1)], axis=1, keys= ['Event a', 'Event b', 'Event before'])

In [ ]:
df_country

## Weekday split

In [ ]:
def plot_weekday_percentage(df, title):
    # order of days
    weekday_order = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday","Sunday"]
    
    # create a new dataframe with the counts of opened_shopping_cart events by weekday
    df_opened_counts = df[df['event_type'] == 'opened_shopping_cart'].groupby('weekday').size()

    # create a new dataframe with the counts of purchased_shopping_cart events by weekday
    df_purchased_counts = df[df['event_type'] == 'purchased_shopping_cart'].groupby('weekday').size()

    # calculate the percentage of opened_shopping_cart events on total opened_shopping_cart by weekday
    df_opened_pct = df_opened_counts / df_opened_counts.sum()

    # calculate the percentage of purchased_shopping_cart events on total purchased_shopping_cart by weekday
    df_purchased_pct = df_purchased_counts / df_purchased_counts.sum()

    # concatenate the two percentage dataframes
    df_concat = pd.concat([df_opened_pct, df_purchased_pct], axis=1)
    df_concat = df_concat.reindex(weekday_order)

    # create the bar plot with a size of 10 inches by 6 inches
    fig, ax = plt.subplots(figsize=(10, 6))
    df_concat.plot(kind='bar', ax=ax)

    # set the title and axis labels
    ax.set_title(title)
    ax.set_xlabel('Weekday')
    ax.set_ylabel('Percentage')

    # add a legend
    ax.legend(['opened_shopping_cart', 'purchased_shopping_cart'])

In [ ]:
plot_weekday_percentage(df_event_a, title="Event A")
plot_weekday_percentage(df_event_b, title="Event B")
plot_weekday_percentage(df_event_before, title="Event Before")

## Sales

In [ ]:
weekday_average = df_event_before.groupby('weekday')['order_value'].mean()
weekday_average

In [ ]:
sales_a = df_event_a.groupby('weekday')['order_value'].mean()
sales_b = df_event_b.groupby('weekday')['order_value'].mean()
sales_before = df_event_before.groupby('weekday')['order_value'].mean()


In [ ]:
# Define the desired order of weekdays
weekday_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

# Reindex the three series using the weekday_order list
sales_a = sales_a.reindex(weekday_order)
sales_b = sales_b.reindex(weekday_order)
sales_before = sales_before.reindex(weekday_order)

# Round the three series to 2 decimal places
sales_a = sales_a.round(2)
sales_b = sales_b.round(2)
sales_before = sales_before.round(2)


In [ ]:
# Combine the three series into one DataFrame
comb_df = pd.DataFrame({'sales_a': sales_a, 'sales_b': sales_b, 'sales_before': sales_before})
comb_df['mean_per_row'] = comb_df.mean(axis=1)
comb_df.mean_per_row = comb_df.mean_per_row.round(2)
# Print the resulting DataFrame
print(comb_df)

### Plot Order Value figures

In [ ]:
fig, ax = plt.subplots(figsize=(15, 10))
plt.ylim((100, comb_df.sales_a.max()+50))
sns.lineplot(comb_df, ax=ax)

### Mean order value

In [ ]:
print(round(df_event_a.order_value.mean(),1))
print(round(df_event_b.order_value.mean(),1))
print(round(df_event_before.order_value.mean(),1))

### Median order value

In [ ]:
print(round(df_event_a.order_value.median(),1))
print(round(df_event_b.order_value.median(),1))
print(round(df_event_before.order_value.median(),1))

### Min and Max order value

In [ ]:
print(f"{df_event_a.order_value.min()} {df_event_a.order_value.max()}")
print(f"{df_event_b.order_value.min()} {df_event_b.order_value.max()}")
print(f"{df_event_before.order_value.min()} {df_event_before.order_value.max()}")

# Looking at gender and purchases changes

In [ ]:
def plot_weekday_gender_purchase(df, title):
    weekday_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

    # Filter the DataFrame to only include purchase_shopping_cart events
    df_purchase = df.loc[df['event_type'] == 'purchased_shopping_cart']

    # Calculate the count of male and female users for each weekday
    df_gender = df_purchase.groupby(['weekday', 'gender']).size().reset_index(name='count')

    # Calculate the percentage of each gender type for each weekday
    df_gender_pct = pd.DataFrame()
    for weekday in df_gender['weekday'].unique():
        df_gender_weekday = df_gender.loc[df_gender['weekday'] == weekday].copy()
        total_count = df_gender_weekday['count'].sum()
        df_gender_weekday['percentage'] = df_gender_weekday['count'] / total_count * 100
        df_gender_pct = pd.concat([df_gender_pct, df_gender_weekday])

    # Create the bar plot

    # Specify the order for the legend labels
    hue_order = ['man', 'woman', 'non-binary']
    
    # Specify the colors for the genders
    colors = {"man": "#339FFF", "woman": "#FF5733", "non-binary" : "#33FF80"}
    
    #Figsize
    fig, ax = plt.subplots(figsize=(10, 5))
    # Create the bar plot
    sns.barplot(data=df_gender_pct, x='weekday', y='percentage', hue='gender', hue_order=hue_order, order=weekday_order, palette=colors, ax=ax)
    sns.color_palette("pastel")
    # Set the title and labels for the plot
    ax.set_title(title)
    #sns.set(title=title, xlabel='Weekday', ylabel='Percentage of Purchases')

    ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.1), ncol=2)
    
    # Show the plot
    plt.show()

In [ ]:
plot_weekday_gender_purchase(df_event_a, title="Event A")
plot_weekday_gender_purchase(df_event_b, title="Event B")
plot_weekday_gender_purchase(df_event_before, title="Event Before")

# Account type

### Event A

In [ ]:
df_event_a.account_type.unique()

In [ ]:
account_event_a = df_event_a.groupby(['account_type','event_type']).count()
account_event_a.rename(columns={'user_id':'Event a'}, inplace=True)
#account_event_a.user_id

In [ ]:
print(f"Basic {round(account_event_a['Event a'][1]/account_event_a['Event a'][0]*100,1)} %")
print(f"Free {round(account_event_a['Event a'][3]/account_event_a['Event a'][2]*100,1)} %")
print(f"Premium {round(account_event_a['Event a'][5]/account_event_a['Event a'][4]*100,1)} %")

In [ ]:
temp_event_a = round(account_event_a.iloc[1::2, :1] / account_event_a.iloc[1::2, :1].sum() *100,1)

### Event B

In [ ]:
account_event_b = df_event_b.groupby(['account_type','event_type']).count()
account_event_b.rename(columns={'user_id':'Event b'}, inplace=True)


In [ ]:
print(f"Basic {round(account_event_b['Event b'][1]/account_event_b['Event b'][0]*100,1)} %")
print(f"Free {round(account_event_b['Event b'][3]/account_event_b['Event b'][2]*100,1)} %")
print(f"Premium {round(account_event_b['Event b'][5]/account_event_b['Event b'][4]*100,1)} %")

In [ ]:
temp_event_b = round(account_event_b.iloc[1::2, :1] / account_event_b.iloc[1::2, :1].sum() *100,1)

### Event Before

In [ ]:
account_event_before = df_event_before.groupby(['account_type','event_type']).count()
account_event_before.rename(columns={'user_id':'Event before'}, inplace=True)
#account_event_before.user_id

In [ ]:
print(f"Basic {round(account_event_before['Event before'][1]/account_event_before['Event before'][0]*100,1)} %")
print(f"Free {round(account_event_before['Event before'][3]/account_event_before['Event before'][2]*100,1)} %")
print(f"Premium {round(account_event_before['Event before'][5]/account_event_before['Event before'][4]*100,1)} %")

In [ ]:
temp_event_before = round(account_event_before.iloc[1::2, :1] / account_event_before.iloc[1::2, :1].sum() *100,1)

In [ ]:
df = pd.concat([temp_event_a, temp_event_b, temp_event_before], axis=1)

In [ ]:
df